In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Read the data

In [3]:
movies = pd.read_csv('ContentBasedRecommenderSystem\movies.csv')
ratings = pd.read_csv('ContentBasedRecommenderSystem/ratings.csv')

In [4]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
ratings


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


# Filter first 200 user and movies


In [40]:
# Filter first 200 user and movies

rating_filter = ratings[ratings['userId']<=200]
top_movies = rating_filter['movieId'].value_counts().index.tolist()
required_ids = [1, 4]
selected_movies = list(dict.fromkeys(required_ids + top_movies))[:200]  # ensures 1 & 4 are included
rating_filter = rating_filter[rating_filter['movieId'].isin(selected_movies)]

In [41]:
# user movie matrix
user_movie_matrix = rating_filter.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_movie_matrix

movieId,1,2,4,6,10,11,19,21,25,32,...,8961,33794,44191,48516,54286,58559,68954,72998,79132,91529
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,4.0,3.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## user movie user matrix


In [42]:
# user movie user matrix
movie_user_matrix = user_movie_matrix.T
movie_user_matrix

userId,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0
10,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,...,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58559,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0
68954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
72998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## cosine similarity between movies


In [43]:
# cosine similarity between movies

similarity_matrix = cosine_similarity(movie_user_matrix)
similarity_df = pd.DataFrame(similarity_matrix, index=movie_user_matrix.index, columns=movie_user_matrix.index)

similarity_df

movieId,1,2,4,6,10,11,19,21,25,32,...,8961,33794,44191,48516,54286,58559,68954,72998,79132,91529
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.364634,0.000000,0.327928,0.352899,0.245691,0.383177,0.314843,0.305103,0.401293,...,0.405373,0.352153,0.256481,0.237384,0.254860,0.268509,0.246652,0.228129,0.244432,0.250658
2,0.364634,1.000000,0.102937,0.278936,0.342590,0.334386,0.344587,0.219876,0.097983,0.296544,...,0.263135,0.229658,0.216003,0.274432,0.174412,0.266757,0.307064,0.321419,0.254434,0.268707
4,0.000000,0.102937,1.000000,0.176604,0.129040,0.195764,0.104069,0.149025,0.289178,0.134383,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.327928,0.278936,0.176604,1.000000,0.304328,0.358692,0.281046,0.438093,0.479313,0.459077,...,0.309837,0.263004,0.271452,0.256726,0.223022,0.188871,0.152767,0.226812,0.192702,0.231025
10,0.352899,0.342590,0.129040,0.304328,1.000000,0.345764,0.352139,0.219544,0.239440,0.254330,...,0.243056,0.203445,0.194677,0.130849,0.236928,0.140298,0.144320,0.149755,0.105132,0.138895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58559,0.268509,0.266757,0.000000,0.188871,0.140298,0.062683,0.257694,0.034781,0.049383,0.176706,...,0.466501,0.673229,0.499881,0.610747,0.602973,1.000000,0.693308,0.586659,0.814708,0.754310
68954,0.246652,0.307064,0.000000,0.152767,0.144320,0.052342,0.238604,0.016667,0.041438,0.134914,...,0.461801,0.502948,0.461982,0.519289,0.467362,0.693308,1.000000,0.728059,0.679805,0.745366
72998,0.228129,0.321419,0.000000,0.226812,0.149755,0.071493,0.240706,0.055634,0.064538,0.183220,...,0.485844,0.489958,0.539230,0.501235,0.480195,0.586659,0.728059,1.000000,0.589710,0.526804


# Get top 10 similar movies


In [44]:
# get top 10 similar movies
def get_top_similar_movies(movie_id, n=10):
    sim_scores = similarity_df[movie_id].drop(movie_id)
    sim_scores = sim_scores.sort_values(ascending=False).head(n).index
    return movies[movies['movieId'].isin(sim_scores)][['movieId', 'title']].reset_index(drop=True)

In [45]:
# a. Top 10 similar to Toy Story (1995) (ID: 1)
print("Top 10 similar to Toy Story (1995):")
print(get_top_similar_movies(1))

Top 10 similar to Toy Story (1995):
   movieId                                      title
0      260  Star Wars: Episode IV - A New Hope (1977)
1      296                        Pulp Fiction (1994)
2      367                           Mask, The (1994)
3      480                       Jurassic Park (1993)
4      588                             Aladdin (1992)
5      648                 Mission: Impossible (1996)
6      780       Independence Day (a.k.a. ID4) (1996)
7     1265                       Groundhog Day (1993)
8     3114                         Toy Story 2 (1999)
9     4306                               Shrek (2001)


In [47]:
# b. Top 10 similar to Waiting to Exhale (1995) (ID: 4)
print("\nTop 10 similar to Waiting to Exhale (1995):")
print(get_top_similar_movies(4))


Top 10 similar to Waiting to Exhale (1995):
   movieId                               title
0       25            Leaving Las Vegas (1995)
1       36             Dead Man Walking (1995)
2       62           Mr. Holland's Opus (1995)
3       95                 Broken Arrow (1996)
4      150                    Apollo 13 (1995)
5      266          Legends of the Fall (1994)
6      337  What's Eating Gilbert Grape (1993)
7      539         Sleepless in Seattle (1993)
8      587                        Ghost (1990)
9      590           Dances with Wolves (1990)


# Recommend 3 movies to user 200


In [48]:
# Recommend 3 movies to user 200
def recommend_to_user(user_id, n=3):
    user_ratings = user_movie_matrix.loc[user_id]
    unrated_movies = user_ratings[user_ratings == 0].index

    predicted_ratings = {}
    for movie in unrated_movies:
        similar_movies = similarity_df[movie]
        rated_movies = user_ratings[user_ratings > 0]
        sim_scores = similar_movies[rated_movies.index]
        if sim_scores.sum() == 0:
            predicted_rating = 0
        else:
            predicted_rating = np.dot(sim_scores, rated_movies) / sim_scores.sum()
        predicted_ratings[movie] = predicted_rating

    top_movies = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:n]
    top_movie_ids = [m[0] for m in top_movies]
    return movies[movies['movieId'].isin(top_movie_ids)][['movieId', 'title']]


In [49]:
print("\nRecommended movies for user 200:")
print(recommend_to_user(200))


Recommended movies for user 200:
      movieId                       title
2945     3949  Requiem for a Dream (2000)
6315    48516        Departed, The (2006)
7372    79132            Inception (2010)
